data sources:

- https://fdc.nal.usda.gov/download-datasets.html
- https://www.ers.usda.gov/data-products/fruit-and-vegetable-prices.aspx

- https://www.kaggle.com/datasets/thedevastator/the-nutritional-content-of-food-a-comprehensive
- https://www.kaggle.com/datasets/trolukovich/nutritional-values-for-common-foods-and-products
- https://www.kaggle.com/datasets/thunderz/food-data


inspiration sources:

- https://nutritionj.biomedcentral.com/articles/10.1186/s12937-019-0496-5
- https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2954450/
- https://www.healthline.com/nutrition/29-cheap-healthy-foods
- https://nutritionfacts.org/blog/best-nutrition-bang-for-your-buck/

In [1]:
import numpy as np
import pandas as pd
import psycopg2
from psycopg2 import Error
import time
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def query(sql):
    try:
        # Connect to an existing database
        connection = psycopg2.connect(user="postgres",
                                      password="PGAdmin123",
                                      host="localhost",
                                      port="5432",
                                      database="Food")
        cursor = connection.cursor()
        cursor.execute(sql)
        records = cursor.fetchall()
        field_names = [i[0] for i in cursor.description]
        df = pd.DataFrame(data = records, columns = field_names)
        return df
    except (Exception, Error) as error:
        #print("Error while connecting to PostgreSQL", error)
        pass
    finally:
        if (connection):
            cursor.close()
            connection.close()

In [3]:
sql = '''
SELECT
    table_name,
    column_name,
    ordinal_position
FROM information_schema.columns
WHERE table_schema = 'public'
'''

#     '''
# SELECT *
# FROM acquisition_samples
# LIMIT 10
# '''

In [4]:
df = query(sql)
df.head()

table_name                 column_name  \
0  food_nutrient_conversion_factor                          id   
1              acquisition_samples  fdc_id_of_acquisition_food   
2             agricultural_samples                      fdc_id   
3             agricultural_samples            acquisition_date   
4  food_nutrient_conversion_factor                      fdc_id   

   ordinal_position  
0                 1  
1                 2  
2                 1  
3                 2  
4                 2

In [5]:
search = df[df['column_name'].apply(lambda x: 'id' in x)][['table_name', 'column_name']]
search.sort_values(['table_name', 'column_name'], inplace=True)
search.reset_index(drop=True)

table_name                         column_name
0               acquisition_samples          fdc_id_of_acquisition_food
1               acquisition_samples               fdc_id_of_sample_food
2              agricultural_samples                              fdc_id
3                              food                              fdc_id
4                              food                    food_category_id
5                    food_attribute                              fdc_id
6                    food_attribute              food_attribute_type_id
7                    food_attribute                                  id
8    food_calorie_conversion_factor  food_nutrient_conversion_factor_id
9                     food_category                                  id
10                   food_component                              fdc_id
11                   food_component                                  id
12                    food_nutrient                       derivation_id
13                    food_nutrient                              fdc_id
14                    food_nutrient                                  id
15                    food_nutrient                         nutrient_id
16  food_nutrient_conversion_factor                              fdc_id
17  food_nutrient_conversion_factor                                  id
18                     food_portion                              fdc_id
19                     food_portion                                  id
20                     food_portion                     measure_unit_id
21   food_protein_conversion_factor  food_nutrient_conversion_factor_id
22            food_update_log_entry                                  id
23                  foundation_food                              fdc_id
24                       input_food                              fdc_id
25                       input_food                                  id
26                       lab_method                                  id
27                  lab_method_code                                  id
28                  lab_method_code                       lab_method_id
29              lab_method_nutrient                                  id
30              lab_method_nutrient                       lab_method_id
31              lab_method_nutrient                         nutrient_id
32               market_acquisition                              fdc_id
33                     measure_unit                                  id
34                         nutrient                                  id
35                      sample_food                              fdc_id
36                  sub_sample_food                              fdc_id
37                  sub_sample_food               fdc_id_of_sample_food
38                sub_sample_result                    food_nutrient_id
39                sub_sample_result                       lab_method_id

In [6]:
# loop thru df and add column of the distinct count of records for each table-column combo
search['DistinctRecords'] = 0
for i in search.index:
    t = search['table_name'][i]
    c = search['column_name'][i]
    q = f"SELECT DISTINCT COUNT({c}) FROM {t}"
    out = query(q)
    search.loc[i, 'DistinctRecords'] = out.iloc[0,0]
search

table_name                         column_name  \
1               acquisition_samples          fdc_id_of_acquisition_food   
65              acquisition_samples               fdc_id_of_sample_food   
2              agricultural_samples                              fdc_id   
34                             food                              fdc_id   
35                             food                    food_category_id   
53                   food_attribute                              fdc_id   
72                   food_attribute              food_attribute_type_id   
52                   food_attribute                                  id   
54   food_calorie_conversion_factor  food_nutrient_conversion_factor_id   
58                    food_category                                  id   
83                   food_component                              fdc_id   
60                   food_component                                  id   
12                    food_nutrient                       derivation_id   
8                     food_nutrient                              fdc_id   
7                     food_nutrient                                  id   
9                     food_nutrient                         nutrient_id   
4   food_nutrient_conversion_factor                              fdc_id   
0   food_nutrient_conversion_factor                                  id   
27                     food_portion                              fdc_id   
26                     food_portion                                  id   
30                     food_portion                     measure_unit_id   
39   food_protein_conversion_factor  food_nutrient_conversion_factor_id   
41            food_update_log_entry                                  id   
37                  foundation_food                              fdc_id   
44                       input_food                              fdc_id   
43                       input_food                                  id   
51                       lab_method                                  id   
46                  lab_method_code                                  id   
47                  lab_method_code                       lab_method_id   
48              lab_method_nutrient                                  id   
49              lab_method_nutrient                       lab_method_id   
50              lab_method_nutrient                         nutrient_id   
5                market_acquisition                              fdc_id   
6                      measure_unit                                  id   
17                         nutrient                                  id   
20                      sample_food                              fdc_id   
21                  sub_sample_food                              fdc_id   
22                  sub_sample_food               fdc_id_of_sample_food   
23                sub_sample_result                    food_nutrient_id   
25                sub_sample_result                       lab_method_id   

    DistinctRecords  
1              7186  
65             7186  
2               810  
34            54960  
35            54142  
53             3891  
72             3891  
52             3891  
54              261  
58               28  
83                0  
60             3066  
12           135052  
8            135073  
7            135073  
9            135073  
4             11268  
0             11268  
27            10678  
26            10678  
30            10678  
39              237  
41            54960  
37              236  
44             4863  
43             4863  
51              275  
46              292  
47              292  
48             1005  
49             1005  
50             1005  
5              6402  
6               122  
17              474  
20             2904  
21            44537  
22            44537  
23           103581  
25           103581

In [7]:
#next run SQL left and inner joins on all table column combos, return record counts

joins = pd.DataFrame(columns=['left_table', 'left_column', 'right_table',
                              'right_column', 'dist_records_left','dist_records_inner'])

s = time.time()


for i in search.index:
    t = search.loc[i, 'table_name']
    c = search.loc[i, 'column_name']

    for j in search.index:
        if j == i:
            pass
        else:
            x = search.loc[j, 'table_name']
            y = search.loc[j, 'column_name']
            if t == x:
                pass
            else:
                try:
                    q = f"SELECT DISTINCT COUNT({t}.{c}) FROM {t} LEFT JOIN {x} ON {t}.{c}::int = {x}.{y}::int"
                    out = query(q)
                    q = f"SELECT DISTINCT COUNT({t}.{c}) FROM {t} INNER JOIN {x} ON {t}.{c}::int = {x}.{y}::int"
                    out2 = query(q)
                    l = [t, c, x, y, out.iloc[0,0], out2.iloc[0,0]]
                    joins.loc[len(joins)] = l
                except:
                    pass

e = time.time()
print(f"loaded in {str(round(e-s,1))+' seconds' if (e-s)<60 else str(round((e-s)/60,1)) + ' minutes'}") 

joins.head()

loaded in 5.3 minutes


left_table                 left_column           right_table  \
0  acquisition_samples  fdc_id_of_acquisition_food  agricultural_samples   
1  acquisition_samples  fdc_id_of_acquisition_food                  food   
2  acquisition_samples  fdc_id_of_acquisition_food                  food   
3  acquisition_samples  fdc_id_of_acquisition_food        food_attribute   
4  acquisition_samples  fdc_id_of_acquisition_food        food_attribute   

       right_column  dist_records_left  dist_records_inner  
0            fdc_id               7186                   0  
1            fdc_id               7186                7186  
2  food_category_id               7186                   0  
3            fdc_id               7568                 597  
4                id               7186                   0

In [8]:
#compare those values to the original records as a percentage
#those with the highest percentage are likely to be legit joins

In [9]:
joins.head()

left_table                 left_column           right_table  \
0  acquisition_samples  fdc_id_of_acquisition_food  agricultural_samples   
1  acquisition_samples  fdc_id_of_acquisition_food                  food   
2  acquisition_samples  fdc_id_of_acquisition_food                  food   
3  acquisition_samples  fdc_id_of_acquisition_food        food_attribute   
4  acquisition_samples  fdc_id_of_acquisition_food        food_attribute   

       right_column  dist_records_left  dist_records_inner  
0            fdc_id               7186                   0  
1            fdc_id               7186                7186  
2  food_category_id               7186                   0  
3            fdc_id               7568                 597  
4                id               7186                   0

In [10]:
results = (joins
    .merge(search,
            left_on=['left_table','left_column'],
            right_on=['table_name', 'column_name'],
            how='left'
           )    
)

results.head()

left_table                 left_column           right_table  \
0  acquisition_samples  fdc_id_of_acquisition_food  agricultural_samples   
1  acquisition_samples  fdc_id_of_acquisition_food                  food   
2  acquisition_samples  fdc_id_of_acquisition_food                  food   
3  acquisition_samples  fdc_id_of_acquisition_food        food_attribute   
4  acquisition_samples  fdc_id_of_acquisition_food        food_attribute   

       right_column  dist_records_left  dist_records_inner  \
0            fdc_id               7186                   0   
1            fdc_id               7186                7186   
2  food_category_id               7186                   0   
3            fdc_id               7568                 597   
4                id               7186                   0   

            table_name                 column_name  DistinctRecords  
0  acquisition_samples  fdc_id_of_acquisition_food             7186  
1  acquisition_samples  fdc_id_of_acquisition_food             7186  
2  acquisition_samples  fdc_id_of_acquisition_food             7186  
3  acquisition_samples  fdc_id_of_acquisition_food             7186  
4  acquisition_samples  fdc_id_of_acquisition_food             7186

In [18]:
results['percent_of_orig'] = results['dist_records_left'] / results['DistinctRecords']

results.sort_values('percent_of_orig',ascending=False)

left_table       left_column        right_table  \
158                  food  food_category_id      food_nutrient   
415         food_nutrient     derivation_id               food   
308         food_category                id      food_nutrient   
1104  lab_method_nutrient     lab_method_id  sub_sample_result   
302         food_category                id               food   
...                   ...               ...                ...   
369        food_component            fdc_id        sample_food   
370        food_component            fdc_id    sub_sample_food   
371        food_component            fdc_id    sub_sample_food   
372        food_component            fdc_id  sub_sample_result   
373        food_component            fdc_id  sub_sample_result   

               right_column  dist_records_left  dist_records_inner  \
158           derivation_id         1201489669          1201447476   
415        food_category_id         1201448710          1201447476   
308           derivation_id             133844              133818   
1104          lab_method_id            3041020             3040588   
302        food_category_id              54152               54142   
...                     ...                ...                 ...   
369                  fdc_id                  0                   0   
370                  fdc_id                  0                   0   
371   fdc_id_of_sample_food                  0                   0   
372        food_nutrient_id                  0                   0   
373           lab_method_id                  0                   0   

               table_name       column_name  DistinctRecords  percent_of_orig  
158                  food  food_category_id            54142     22191.453382  
415         food_nutrient     derivation_id           135052      8896.193392  
308         food_category                id               28      4780.142857  
1104  lab_method_nutrient     lab_method_id             1005      3025.890547  
302         food_category                id               28      1934.000000  
...                   ...               ...              ...              ...  
369        food_component            fdc_id                0              NaN  
370        food_component            fdc_id                0              NaN  
371        food_component            fdc_id                0              NaN  
372        food_component            fdc_id                0              NaN  
373        food_component            fdc_id                0              NaN  

[1441 rows x 10 columns]

In [19]:
# forget this ... this is becoming too complicated to deal with Postgres tables, looping to find 
# primary keys. Just read the documentation, understand the data set, and just pd.read_csv files in 
# and perform EDA...